# Run the final state algorithm on a ROOT HSDATA file

First create an object of your final state class. You may configure it if you want to limit the number of particles in an event or set to just run on generated simulation data.
This notebook is equivalent to the RunFSRootXXX.C macro which will be in the code directory.
Again just change XXX for your class name.
The string arguements set which particles you would like to use their pre-determined PID for and which you would like to be inclusive (i.e. any number of thse particles).
**Example**, 
1. for tagged photon experiment I might want fs("Beam","Beam")
This means use predetermined photon ID (i.e. I know it came from the tagger) and I will analyse events with any number of tagger photons.
2. fs("e-","proton:pi+:pi-")  i.e. use predetemined electron PID and just determine hadron species based on charge and try all combinations in events with any number of proton,pi+,pi-, but with the number of electrons defined for the topology
3. fs("NONE","ALL") => do not use any PID info other than charge, analyse events with any number of particles (this can be limited by SetMaxParticles)

In [ ]:
gROOT->ProcessLine(".x LoadXXX.C+"); //Load the classes

In [ ]:
 XXX fs("NONE","ALL"); //string arguments => PID, INCLUSIVE
  // fs.SetGenerated(); //just analyse generated branch
  // fs.SetMaxParticles(10); //max number of particles of any 1 type
 

Create an output tree. Here we will use a FiledTree object which is a wrapper class to a tree in a file. We will also create this as a smart pointer so we can delete it at the end by calling reset(), this will save the tree to disk. The Filed tree needs 2 strings ,1 a treename and the other the file name. You should give the full path for where you would like the files saved. It will overwrite any existing files. 

In [ ]:
  //strings = treename and filename (give full path)
  auto output = new FiledTree("FinalTree","OutPUT.root");
  fs.FinalStateOutTree(output->Tree()); //connect ouput tree to final state branches

Now we configure the input data. Here we use a ROOT file presaved in HSDATA format. It is possible to analyse other dataformats via the DataManager class.

In [ ]:
 //create datamanager
  auto dm=std::make_shared<DataManager>();

  //And make a chain of data files
  TChain chain("HSParticles");
  chain.Add("/indir/out_*root");
  dm->InitChain(&chain);

  //connect FinalState to Data by moving the pointer
  fs.SetDataManager(dm);

Analyse all the events. Not you may prefer just looping over the data manager directly via while(dm->ReadEvent()) fs->Process();
You can also supply a number of events to analyse e.g. fs.ProcessData(100);

In [ ]:
cout<<"Number of Events to process "<<chain.GetEntries()<<endl;
fs.ProcessData(); //No number give, analyse all events in chain

Draw some variables from your tree.

In [ ]:
TCanvas c1;
output->Tree()->Draw("MissMass>>(100,-1,2)","Topo==0");
//output->Tree()->Draw("MissMass","Topo==1","same");
//output->Tree()->Draw("MissMass","Topo==2","same");
//output->Tree()->Draw("MissMass","Topo==3","same");
c1.Draw();

And remember to save the tree!

In [ ]:
delete output;